Imports

In [ ]:
from bs4 import BeautifulSoup, Tag
import requests
import pprint
!pip install stanza
import re

In [2]:
import pandas as pd
import stanza

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import glob
import pandas as pd
en_files = sorted(glob.glob('/content/drive/MyDrive/Thesis2023/First1K_XML/*1st1K-grc1.xml'))  #creates a list of files based on filepath and regular expressions
en_files   #useful for automating changes to a large number of files


In [4]:
import os

In [41]:
for f in en_files: #renaming files
  old_file_name = f
  new_file_name = old_file_name.replace('1st1K-grc1.xml', '.1st1K-grc1.xml')
  os.rename(old_file_name, new_file_name)



In [ ]:
import glob
import pandas as pd
grc_files = sorted(glob.glob('/content/drive/MyDrive/Thesis2023/First1K_XML/*.xml'))  #creates a list of files based on filepath and regular expressions
grc_files   #useful for automating changes to a large number of files
#grc_colson=

XML Edits

In [5]:
#loading the file with beautiful soup
def open_file(x):
  philo = open( x ,encoding='utf-8', errors='ignore')
  philo_soup = BeautifulSoup(philo,'lxml')
  return philo_soup



In [9]:
#Four of Philo's works (tlg0018.tlg002,tlg0018.tlg019, tlg0018.tlg022, tlg0018.tlg024) have books and sections
#So the function for changing those XML:Base tags is slightly different
def making_books_and_secs(philo_soup):
  for b in philo_soup.find_all('div', subtype= 'book'):
    b['xml:base']= b['xml:base']+':'+(str(b['n'])) #grabs every heading for each book
    for sec in b.find_all('div', subtype= 'section'): #accesses a list of section headings that occur under each book
      sec['xml:base']= b['xml:base']+ '.'+(str(sec['n']))  #Beautiful soup stores XML tags as dictionaries- so we can access and change them the way we would a python dictionary
  return philo_soup

In [7]:
#This function was used for all the works that contain only sections
def make_sections(philo_soup):
  for sec in philo_soup.find_all('div', subtype= 'section'):
    sec['xml:base']= sec['xml:base']+':'+(str(sec['n'])) #accesses existing XML base tag and adds ':' and the n attribute
  return philo_soup                                       #n attribute is made into a string here because beautiful soup won't accept integers


In [6]:
def add_revision_tags(opened):  #editing the TEI header of each file to document my revisions
  original_tag= opened.revisiondesc
  new_tag = opened.new_tag("change", when= "2024-01-01",who="Julia Deen")
  original_tag.append(new_tag)
  original_tag
  new_tag.string = "Updated the xml:base tags to match the most recent CTS URN guidelines specified by the OpenGreekandLatin Github page"
  original_tag

In [7]:
def write_file(philo_soup,x):
  with open(f"{x}", 'w') as philo_file:
    philo_file.write(str(philo_soup)) #writes the XML file to save the changes


In [ ]:
for x in grc_files: #allows us to automate the process by looping through all files in the directory
  opened= open_file(x)
  sected=  making_books_and_secs(opened)
  revised= add_revision_tags(sected)
  written= write_file(revised,x)


In [ ]:
corr_files = sorted(glob.glob('/content/*.xm'l)) #using regular expressions to search file directory
corr_files  #for our corrected files

In [21]:
import zipfile

In [22]:
with zipfile.ZipFile('1st1K_XML.zip', 'w') as f: #creating a zipfile of our now edited files
    for y in corr_files:
        f.write(y)